# Walk-Forward Analysis with Hidden Markov Models

This notebook builds a complete walk-forward backtesting engine for a Market Regime-Switching strategy.

We start from the core GMM-HMM model (Gaussian Mixture Model - Hidden Markov Model) explored in `hmm_model.py` and extend it to a rolling-window framework to realistically simulate historical performance.

## Chapter 1: Data Gathering & Initialization

Our first step is to configure the environment, load the financial data, and inspect it to ensure it is suitable for a long-term walk-forward analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from stock import Stock
from typing import Tuple, List, Dict, Optional

### 1.1 Configuration

We select the asset and the time frequency. For a robust walk-forward test, we need a long history of data. 
We also define the `TRAINING_WINDOW`, which determines how much past data the model 'sees' at each step to learn the regimes.

In [ ]:
# --- Configuration ---
TICKER = 'AAPL'            # Asset to backtest
TIME_FREQUENCY = 'weekly'  # 'daily' or 'weekly'

# Walk-Forward Parameters
# How many periods of history to use for training the HMM at each step
# Dynamic logic: 500 weeks (~10 years) or 1500 days (~6 years)
if TIME_FREQUENCY == 'weekly':
    TRAINING_WINDOW = 500
else:
    TRAINING_WINDOW = 1500

print(f"Configuration: {TICKER} [{TIME_FREQUENCY}]")
print(f"Rolling Window Size: {TRAINING_WINDOW} periods")

### 1.2 Data Loading

We use the project's `Stock` class to fetch data from Yahoo Finance. We ensure we have enough history to support both the initial training window and a significant out-of-sample testing period.

In [ ]:
# Initialize and load data
my_stock = Stock(TICKER, TIME_FREQUENCY)
# Load as much history as possible (e.g. from 1990)
my_stock.load_data(start_date='1980-01-01')

# Extract key series
df = my_stock.get_data()
prices = df['Close']
log_returns = df['Log_Returns'].dropna()

print(f"Data Loaded: {len(log_returns)} observations")
print(f"Date Range: {log_returns.index[0].date()} to {log_returns.index[-1].date()}")

### 1.3 Inspection for Walk-Forward Validity

Before we proceed to modeling, we visualize the full history. 
1. **Price History (Log Scale)**: Verifies the long-term trend and data quality.
2. **Returns Distribution**: Confirms the non-Normal characteristics (fat tails) that justify using an HMM.

In [ ]:
# Visualize Price History
plt.figure(figsize=(12, 6))
plt.semilogy(prices.index, prices.values, label=f'{TICKER} Close')
plt.title(f"{TICKER} Price History (Log Scale)")
plt.ylabel("Price ($)")
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y:.0f}'))
ax.minorticks_off()  # Remove minor tick marks
plt.grid(True, which="major", ls="-", alpha=0.2)
plt.legend()
plt.show()

### Empirical Distribution Analysis

Here we visually examine the distribution of log-returns. The red line represents a standard Normal distribution fitted to the data.
Note the discrepancies: the empirical data has generally a higher peak and much wider "tails" (extreme values) than the Normal curve predicts.
This motivates using a **Mixture Model** (combination of Gaussians) rather than a simple Gaussian distribution.

In [ ]:
def remove_outliers(data: pd.Series, multiplier: int = 4):
    """
    Filter out extreme outliers for cleaner visualization (using IQR method).
    This helps in seeing the main body of the distribution without the graph being distorted by 1-2 extreme crashes.
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    return data[(data >= Q1 - multiplier * IQR) & (data <= Q3 + multiplier * IQR)]

# Filter the returns from outliers ONLY for a better visualization
filtered_returns = remove_outliers(log_returns, multiplier=5)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram vs Normal
sns.histplot(filtered_returns, bins=100, stat='density', alpha=0.6, color='steelblue', label='Empirical Data', ax=ax1)
x_range = np.linspace(filtered_returns.min(), filtered_returns.max(), 100)
stats_mu, stats_sigma = filtered_returns.mean(), filtered_returns.std()
ax1.plot(x_range, stats.norm.pdf(x_range, stats_mu, stats_sigma), 
        'r--', lw=2, label=f'Normal ($\mu$={stats_mu:.4f}, $\sigma$={stats_sigma:.4f})')
ax1.set_title(f'Returns Distribution ({TICKER})')
ax1.legend()

# Q-Q Plot
# Deviations from the red line indicate non-normality (fat tails)
stats.probplot(filtered_returns, dist="norm", plot=ax2)
ax2.get_lines()[0].set_markerfacecolor('steelblue')
ax2.get_lines()[0].set_markersize(4)
ax2.set_title('Q-Q Plot')

plt.tight_layout()
plt.show()

# Quick sanity check for Walk-Forward
total_observations = len(log_returns)
max_cycles = total_observations - TRAINING_WINDOW
if max_cycles <= 0:
    raise ValueError(f"Insufficient data. Need > {TRAINING_WINDOW} periods, but have {total_observations}.")
    
print(f"Data Check: {total_observations} obs. Sufficient for {max_cycles} Out-of-Sample predictions.")

## Chapter 2: The Identification Problem & Economic Sorting

### The Theoretical Challenge: Label Switching

In finite mixture models and Hidden Markov Models, we encounter the **Identification Problem** (often referred to as "Label Switching").
The Likelihood function of the model is invariant to the permutation of the latent states.
This means that if we simply swap the parameters of "State 0" and "State 1" (e.g., exchange their means, variances, and transition probabilities), the model explains the data equally well. The statistical algorithm has no intrinsic concept that "State 0" should represent a Bear market and "State 1" a Bull market.

In the context of a **Walk-Forward (Rolling Window) Analysis**, this is critical.
*   **Window $T$**: The model might identify "State 0" as the Low-Volatility regime and "State 1" as High-Volatility.
*   **Window $T+1$**: The numerical optimization might swap these, identifying "State 0" as High-Volatility.

Without correction, concatenating the state predictions across windows results in an incoherent time series, rendering the backtest invalid.

### Solution: Imposing a Statistical Ordering

To resolve this, we must strictly order the states based on an invariant economic metric—a **Utility Function** $U(\cdot)$—such that for our sorted states:
$$ U(\text{State}_0) < U(\text{State}_1) < \dots < U(\text{State}_{K-1}) $$

We define this function to sort regimes from "Worst" (Bear/Crash) to "Best" (Bull/Stable).

**Why Mean-Variance is usually enough (but we go further)**:
A standard **Mean-Variance** utility is a robust and effective way to distinguish regimes. High returns and low volatility clearly indicate a "Bull" state, while negative returns and high volatility indicate a "Bear" state.

However, adding **Skewness** and **Kurtosis** to the objective function can provide an extra layer of precision. This helps specifically in "close calls"—for example, distinguishing between a volatile recovery (high vol, positive skew) and a volatile crash (high vol, negative skew).

We use a **Higher-Moment Utility Proxy**:

$$ U(s) = \mu_s - \frac{1}{2}\sigma_s^2 + \lambda_1 \cdot \text{Skew}_s - \lambda_2 \cdot \text{Kurt}_s $$

*   **$\mu_s, \sigma_s^2$**: The aggregated mean and variance of the Gaussian Mixture for state $s$.
*   **$\text{Skew}_s$**: Captures asymmetry (Crash Risk).
*   **$\text{Kurt}_s$**: Captures tail thickness (Fat Tails).

In [ ]:
from hmmlearn.hmm import GMMHMM

def calculate_state_moments(model: GMMHMM, state_idx: int) -> Dict[str, float]:
    """
    Calculates the first four moments (Mean, Variance, Skewness, Kurtosis) 
    for a specific state's Gaussian Mixture distribution.

    This function aggregates the parameters of the component Gaussians within a state 
    using the Law of Total Cumulants/Moments.

    :param model: The trained Hidden Markov Model.
    :param state_idx: The index of the hidden state to analyze.

    :return: A dictionary containing:
        - 'mean': Aggregated expected return.
        - 'var': Aggregated variance.
        - 'skew': Fisher-Pearson coefficient of skewness.
        - 'kurt': Excess kurtosis.
    """
    # 1. Extract mixture parameters for the specific state
    weights = model.weights_[state_idx]
    means = model.means_[state_idx].flatten()
    
    # Handle covariance types
    if model.covariance_type == 'tied':
        # One shared covariance matrix for all states/components
        covs = np.full(len(weights), model.covars_[state_idx][0][0])
    elif model.covariance_type == 'diag':
        # Diagonal covariance per component
        covs = model.covars_[state_idx].flatten()
    else:
        # Full covariance, extracting diagonal elements for 1D approximation
        covs = model.covars_[state_idx].flatten()

    # 2. Aggregated Mean: E[X]
    # Simple weighted sum of component means
    agg_mean = np.sum(weights * means)

    # 3. Aggregated Variance: Law of Total Variance
    # Var(X) = E[Var(X|Z)] + Var(E[X|Z])
    #        = sum(w * sigma^2) + sum(w * (mu - agg_mean)^2)
    agg_var = np.sum(weights * covs) + np.sum(weights * (means - agg_mean)**2)
    
    # Numerical stability check
    if agg_var < 1e-12:
        return {'mean': agg_mean, 'var': 0.0, 'skew': 0.0, 'kurt': 0.0}
    
    # 4. Higher Moments via Central Moments
    # Deviation of component means from aggregate mean
    d = means - agg_mean
    
    # 3rd Central Moment (Skewness numerator): E[(X - mu)^3]
    # approx: sum(w * (d^3 + 3*d*sigma^2))
    m3 = np.sum(weights * (d**3 + 3 * d * covs))
    
    # 4th Central Moment (Kurtosis numerator): E[(X - mu)^4]
    # approx: sum(w * (d^4 + 6*d^2*sigma^2 + 3*sigma^4))
    m4 = np.sum(weights * (d**4 + 6 * (d**2) * covs + 3 * (covs**2)))
    
    # Standardize
    skew = m3 / (agg_var ** 1.5)
    kurt = (m4 / (agg_var ** 2)) - 3.0  # Excess kurtosis

    return {
        'mean': agg_mean,
        'var': agg_var,
        'skew': skew,
        'kurt': kurt
    }

def get_utility_score(
    moments: Dict[str, float], 
    skew_weight: float = 0.5, 
    kurt_weight: float = 0.1
) -> float:
    """
    Computes a risk-adjusted utility score from distributional moments.
    
    Score = Mean - 0.5 * Variance + Skew_Penalty - Kurt_Penalty

    :param moments: Dictionary with 'mean', 'var', 'skew', 'kurt'.
    :param skew_weight: Weight for skewness preference. Positive value rewards positive skew.
    :param kurt_weight: Weight for kurtosis penalty. Positive value penalizes fat tails.

    :return: The computed utility score.
    """
    # 1. Mean-Variance Base (Certainty Equivalent approximation)
    score = moments['mean'] - 0.5 * moments['var']
    
    # 2. Skewness Adjustment
    # We penalize negative skewness (crash risk). 
    # If skew is positive, we add to score. If negative, we subtract.
    score += skew_weight * moments['skew']
    
    # 3. Kurtosis Adjustment
    # We penalize excess kurtosis (uncertainty/fat tails).
    # Higher kurtosis reduces the score.
    score -= kurt_weight * moments['kurt']
    
    return score

def sort_states_by_utility(
    model: GMMHMM, 
    skew_weight: float = 0.25, 
    kurt_weight: float = 0.10
) -> np.ndarray:
    """
    Determines the sorting permutation that ranks states from Lowest Utility to Highest Utility.

    :param model: The trained HMM model.
    :param skew_weight: Importance of skewness in utility.
    :param kurt_weight: Importance of kurtosis in utility.

    :return: An array of indices representing the sorted order.
             e.g. [2, 0, 1] means State 2 is Worst, State 0 is Neutral, State 1 is Best.
    """
    n_states = model.n_components
    scores = []
    
    for s in range(n_states):
        moments = calculate_state_moments(model, s)
        u = get_utility_score(moments, skew_weight, kurt_weight)
        scores.append(u)
        
    # Return indices that sort the scores in ascending order
    return np.argsort(scores)

def apply_state_permutation(model: GMMHMM, perm: np.ndarray) -> None:
    """
    Reorders the internal parameters of the GMMHMM model in-place according to a permutation.
    
    This ensures that 'State 0' always corresponds to the first index in 'perm',
    'State 1' to the second, etc.

    :param model: The model to modify.
    :param perm: The new order of states (e.g. [2, 0, 1]).
    """
    if np.all(perm == np.arange(len(perm))): 
        return

    # 1. Reorder Initial Probabilities (startprob_)
    model.startprob_ = model.startprob_[perm]

    # 2. Reorder Transition Matrix (transmat_)
    # We must reorder both rows (from_state) and columns (to_state)
    model.transmat_ = model.transmat_[perm][:, perm]

    # 3. Reorder Emission Parameters (weights, means, covars)
    # These have shape (n_comp, n_mix, ...) so we just reorder the first axis
    model.weights_ = model.weights_[perm]
    model.means_ = model.means_[perm]
    model.covars_ = model.covars_[perm]